## Полная проблема собственных значений (метод Якоби).

In [11]:
from numpy import eye, zeros, sign, diag, sort
from math import sqrt
from scipy.linalg import hilbert, eig
from numpy.linalg import norm
from prettytable import PrettyTable

In [12]:
def max_element(A):
    max_i, max_j = 0, 1
    max_elem = abs(A[max_i, max_j])
    n = A.shape[0]
    for i in range(n):
        for j in range(n):
            if i != j:
                if abs(A[i,j]) > max_elem:
                    max_elem = abs(A[i, j])
                    max_i, max_j = i, j
    return max_i, max_j 

In [13]:
def find_cs(A, i, j):
    x = -2 * A[i,j]
    y = A[i,i] - A[j,j]
    if y == 0:
        c = 1/sqrt(2)
        s = c
    else:
        c = sqrt(0.5 * (1 + abs(y) / sqrt(x ** 2 + y ** 2)))
        s = sign(x * y) * abs(x) / (2 * c * sqrt(x ** 2 + y ** 2))
    return c, s

In [14]:
def find_T(n, i, j, c, s):
    T = eye(n)
    T[i, i] = c
    T[i, j] = -s
    T[j, i] = s
    T[j, j] = c
    return T

In [15]:
def jacobi_method(A, eps, strategy="max"):
    iters = 0
    n = A.shape[0]
    i, j = 0, 0
    while True: 
        R_i = []
        for k in range(n):
            R_i.append(sum(abs(A[k])) - abs(A[k,k]))
            
        if len([r for r in R_i if r < eps]) > 0 or iters >= 1000:
            return diag(A), iters 
            
        if strategy == "max":
            i, j = max_element(A)
            
        if strategy == "cyclic":
            if j+1 != i and j+1 != n:
                j += 1
            elif j+1 == i:
                j += 2
            else:
                i += 1
                j = 0
        
        if i == n-1 and j == n:
            i = 0
            j = 1
        
        c, s = find_cs(A, i, j)
        T = find_T(A.shape[0], i, j, c, s)
        A = T @ A @ T.transpose()
        iters += 1

In [16]:
matrices = []

#матрица Гильберта 3х3
matrices.append(hilbert(3))

#матрица Гильберта 5х5
matrices.append(hilbert(5))

#матрица Гильберта 10х10
matrices.append(hilbert(10))

In [17]:
for matrix in matrices:
    results = PrettyTable()
    results.field_names = ["eps", "1 стратегия", "итерации", "2 стратегия", "итерации "]

    
    for eps in range(-16, -1, 2):
        eps = 10**eps
        results.add_row([eps, norm(sort(jacobi_method(matrix, eps)[0])-sort(eig(matrix)[0])), jacobi_method(matrix, eps)[1],
                        norm(sort(jacobi_method(matrix, eps, "cyclic")[0])-sort(eig(matrix)[0])), 
                         jacobi_method(matrix, eps, "cyclic")[1]])
        
    print(results)

+--------+------------------------+----------+------------------------+-----------+
|  eps   |      1 стратегия       | итерации |      2 стратегия       | итерации  |
+--------+------------------------+----------+------------------------+-----------+
| 1e-16  | 4.475527734957445e-16  |    8     | 5.551115123125783e-16  |     8     |
| 1e-14  | 4.475460496224462e-16  |    7     | 5.551115123125783e-16  |     8     |
| 1e-12  | 4.475460496224462e-16  |    7     | 5.551142228113921e-16  |     7     |
| 1e-10  | 4.475460496224462e-16  |    7     | 5.551142228113921e-16  |     7     |
| 1e-08  | 2.8468131512045267e-12 |    6     | 4.2988341510058943e-07 |     6     |
| 1e-06  | 2.8468131512045267e-12 |    6     | 4.298829110036661e-07  |     5     |
| 0.0001 | 3.070874197317984e-07  |    5     | 4.298829110036661e-07  |     5     |
|  0.01  |  0.000198194506945042  |    4     |  0.08046176877363061   |     3     |
+--------+------------------------+----------+------------------------+-----